In [ ]:
from options.options import Options
import os
import torch
from build_dataset_model import build_loaders, build_model
from utils import get_model_attr, calculate_model_losses, tensor_aug
from collections import defaultdict
import math

In [ ]:
args = Options().parse()
if (args.output_dir is not None) and (not os.path.isdir(args.output_dir)):
    os.mkdir(args.output_dir)
if (args.test_dir is not None) and (not os.path.isdir(args.test_dir)):
    os.mkdir(args.test_dir)

In [ ]:
vocab, train_loader, val_loader = build_loaders(args)

In [ ]:
dt = train_loader.dataset

In [ ]:
model, model_kwargs = build_model(args, vocab)

In [ ]:
print(model)
model.float().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate)

In [ ]:
for batch in train_loader:
    ids, objs, boxes, triples, angles, attributes, obj_to_img, triple_to_img = tensor_aug(batch)
    break

In [ ]:
model_out = model(objs, triples, boxes, angles, attributes, obj_to_img)
mu, logvar, boxes_pred, angles_pred = model_out

In [ ]:
boxes.shape

In [ ]:
mu.shape

In [ ]:
dt[1][2].shape

In [ ]:
boxes.shape

In [ ]:
O, T = objs.size(0), triples.size(0)
s, p, o = triples.chunk(3, dim=1)  # All have shape (T, 1)
s, p, o = [x.squeeze(1) for x in [s, p, o]]  # Now have shape (T,)
edges = torch.stack([s, o], dim=1)  # Shape is (T, 2)


In [ ]:
obj_vecs = model.obj_embeddings_ec(objs)
if model.use_attr:
    attr_vecs = model.attr_embedding_ec(attributes)
    obj_vecs = torch.cat([obj_vecs, attr_vecs], dim=1)
angle_vecs = model.angle_embeddings(angles)
pred_vecs = model.pred_embeddings_ec(p)
boxes_vecs = model.box_embeddings(boxes)

In [ ]:
boxes_vecs.shape